In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os
import torch.nn as nn
import torch.optim as optim
from random import shuffle
import wandb
import numpy as np
from torchvision import datasets, models, transforms
from torch.utils.data import SubsetRandomSampler
import torchvision

In [ ]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip

--2024-04-08 05:23:27--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.207, 173.194.210.207, 173.194.211.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: 'nature_12K.zip'

nature_12K.zip      100%[===================>]   3.55G   200MB/s    in 18s     

2024-04-08 05:23:46 (198 MB/s) - 'nature_12K.zip' saved [3816687935/3816687935]



In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.utils.data import SubsetRandomSampler




def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct_predictions / total_predictions
    return epoch_loss, epoch_accuracy


def evaluate(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
    loss = running_loss / len(data_loader.dataset)
    accuracy = correct_predictions / total_predictions
    return loss, accuracy


def fit_model(img_size, n_filters, filter_size, fc_size, drop_out, n_classes, learning_rate, num_epochs,
              activation, optimizer_type):

    criterion = nn.CrossEntropyLoss()
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    train_data_path = "/kaggle/working/inaturalist_12K/train"
    test_data_path = "/kaggle/working/inaturalist_12K/val"
    train_dataset = datasets.ImageFolder(train_data_path, transform=transform)
    val_size = int(0.1 * len(train_dataset))
    class_indices = {class_idx: [] for class_idx in range(len(train_dataset.classes))}
    for idx, (_, label) in enumerate(train_dataset):
        class_indices[label].append(idx)
    val_indices = []
    for class_idx, indices in class_indices.items():
        val_indices.extend(indices[:val_size // len(train_dataset.classes)])
    val_sampler = SubsetRandomSampler(val_indices)
    batch_size = 512
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=val_sampler)

    model = models.resnet50(pretrained=True)
    for param in model.parameters():
      param.requires_grad=False

    num_features=model.fc.in_features
    model.fc=nn.Linear(num_features,10)

    if optimizer_type == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_type == 'nadam':
        optimizer = optim.NAdam(model.parameters(), lr=learning_rate)
    elif optimizer_type == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        train_loss, accuracy = train(model, train_loader, criterion, optimizer, device)
        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {accuracy:.4f}")
        wandb.log({"train_loss":train_loss,"train_acc":accuracy,"Epoch":epoch})

        val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
        wandb.log({"Val_loss":val_loss,"Val_acc":val_accuracy})

    test_dataset = datasets.ImageFolder(test_data_path, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    return model


# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

img_size = 224
n_filters = [16, 32, 64, 128, 256]
filter_size = [3, 3, 3, 3, 3]
fc_size = 512
drop_out = 0.5
n_classes = 10
learning_rate = 0.0001
num_epochs = 10
activation = 'relu'
optimizer_type = 'adam'

# Fit the model
trained_model = fit_model(img_size, n_filters, filter_size, fc_size, drop_out, n_classes, learning_rate, num_epochs,
                     activation, optimizer_type)


Epoch 1/10, Train Loss: 2.2214, Train Accuracy: 0.2256
Validation Loss: 0.2274, Validation Accuracy: 0.1141


In [ ]:
def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")


    # Set up WandB
    if not args.no_wandb:
        wandb.init(project="CS6910_ass2", entity="joel-raj2017")

    # Model hyperparameters
    img_size = args.img_size
    n_filters = args.n_filters
    filter_size = args.filter_size
    fc_size = args.fc_size
    drop_out = args.drop_out
    n_classes = args.n_classes
    learning_rate = args.learning_rate
    num_epochs = args.num_epochs
    activation = args.activation
    optimizer_type = args.optimizer_type

    # Fit the model
    trained_model = fit_model(img_size, n_filters, filter_size, fc_size, drop_out, n_classes, learning_rate, num_epochs,
                              activation, optimizer_type, device)

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Train CNN model for classification")
    parser.add_argument("--img_size", type=int, default=224, help="Input image size")
    parser.add_argument("--n_filters", nargs="+", type=int, default=[16, 32, 64, 128, 256], help="Number of filters in each convolutional layer")
    parser.add_argument("--filter_size", nargs="+", type=int, default=[3, 3, 3, 3, 3], help="Size of filters in each convolutional layer")
    parser.add_argument("--fc_size", type=int, default=512, help="Size of the fully connected layer")
    parser.add_argument("--drop_out", type=float, default=0.5, help="Dropout probability")
    parser.add_argument("--n_classes", type=int, default=10, help="Number of output classes")
    parser.add_argument("--learning_rate", type=float, default=0.0001, help="Learning rate")
    parser.add_argument("--num_epochs", type=int, default=10, help="Number of epochs")
    parser.add_argument("--activation", choices=["relu", "elu", "silu", "selu", "gelu"], default="relu", help="Activation function")
    parser.add_argument("--optimizer_type", choices=["adam", "nadam", "rmsprop"], default="adam", help="Optimizer type")
    parser.add_argument("--no_cuda", action="store_true", help="Disable CUDA")
    parser.add_argument("--no_wandb", action="store_true", help="Disable WandB logging")
    args = parser.parse_args()

    main(args)